<a href="https://colab.research.google.com/github/Leo140102/desafioPython/blob/main/desafioPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import hashlib
import requests
import pandas as pd
from google.colab import userdata

# Chaves de API
PUBLIC_KEY = userdata.get('PUBLIC_KEY')
PRIVATE_KEY = userdata.get('PRIVATE_KEY')

# Criando o hash MD5
hashtre = hashlib.md5(str.encode(str(1) + str(PRIVATE_KEY) + str(PUBLIC_KEY)))

# Base URL da API da Marvel
base = "http://gateway.marvel.com"
url_montada_characters = f"{base}/v1/public/characters?ts=1&apikey={PUBLIC_KEY}&hash={hashtre.hexdigest()}"
url_montada_comics = f"{base}/v1/public/comics?ts=1&apikey={PUBLIC_KEY}&hash={hashtre.hexdigest()}"
url_montada_events = f"{base}/v1/public/events?ts=1&apikey={PUBLIC_KEY}&hash={hashtre.hexdigest()}"

# Fazendo a requisição GET para a API e obtendo os dados
response_characters = requests.get(url_montada_characters)
response_comics = requests.get(url_montada_comics)
response_events = requests.get(url_montada_events)

# Convertendo a resposta para JSON
dados_characters = response_characters.json()
dados_comics = response_comics.json()
dados_events = response_events.json()

# Acessando os resultados dos personagens
descricao_characters = dados_characters["data"]["results"]
descricao_comics = dados_comics["data"]["results"]
descricao_events = dados_events["data"]["results"]